# jaccard 유사도를 활용한 Document 비교

## Library Import

In [262]:
import numpy as np
import pandas as pd

## data 불러오기


In [263]:
df=pd.read_excel('./jaccard_sim.xlsx') # data 불러오기

## data 확인하기

In [264]:
print(df.shape)
df

(15, 2)


,문서,데이터
0,A,이름
1,A,금액
2,A,사진
3,A,지원금
4,A,세금
5,B,이름
6,B,금액
7,B,연락처
8,B,주소
9,C,나이


# 문서 Feature 분류하기

## One hot encoding

In [265]:
df_feat=pd.get_dummies(df['데이터'])
df_feat

,금액,나이,보험금,사진,세금,연락처,월급,이름,주소,지원금
0,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0
6,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,0,1,0,0,0,0,0,0,0,0


## Dataframe 합치기

In [266]:
df_new=pd.concat([df_doc,df_feat],axis=1)
# df_new

## 문서 이름별로 합치기

In [267]:
df_groupby=df_new.groupby('문서').sum() #문서명을 기준으로 Feature의 합계를 구한다
df_groupby

,금액,나이,보험금,사진,세금,연락처,월급,이름,주소,지원금
문서,,,,,,,,,,
A,1,0,0,1,1,0,0,1,0,1
B,1,0,0,0,0,1,0,1,1,0
C,0,1,0,0,0,0,0,1,0,1
D,0,0,1,0,0,0,1,0,1,0


## 문서명 분류하기

In [268]:
doc=df_groupby.index.to_list()
doc

['A', 'B', 'C', 'D']

## feature만 분류하기

In [269]:
data=df_groupby.reset_index().iloc[:,1:] # 기존의 인덱스였던 문서명 제거 및 필요한 칼럼만 뽑아냄
data #결과 확인하기

,금액,나이,보험금,사진,세금,연락처,월급,이름,주소,지원금
0,1,0,0,1,1,0,0,1,0,1
1,1,0,0,0,0,1,0,1,1,0
2,0,1,0,0,0,0,0,1,0,1
3,0,0,1,0,0,0,1,0,1,0


### numpy 행렬로 변환하기

#### 전체 계산한 결과 (A와 B의 교집합)

In [271]:
a=data.to_numpy() #numpy로 변환
b=a.T # a의 전치행렬
a.dot(b) # A,B,C,D 문서별로 일치하는 항목 개수 확인 (내적을 이용하여 중복되는 항목은 1, 아니면 0으로 계산)

array([[5, 2, 2, 0],
       [2, 4, 1, 1],
       [2, 1, 3, 0],
       [0, 1, 0, 3]], dtype=uint8)

### 교집합 계산 결과

In [273]:
df_intersection=pd.DataFrame(a.dot(b),index=doc,columns=doc)
df_intersection # dataframe으로 저장하기

,A,B,C,D
A,5,2,2,0
B,2,4,1,1
C,2,1,3,0
D,0,1,0,3


## jaccard 계산하기 위한 메소드 정의

In [274]:
def solution(arr1, arr2):
    # 두 행렬을 파라미터로 받아서, 조건식 결과값의 형태를 가지고, 모든 값이 0인 행렬을 만든다.
    answer = [len(arr2[0]) * [0] for i in range(len(arr1))]
    
    # 정답 행렬을 2중 for문을 통해 순회하면서, 각 요소의 값을 채운다.
    for i in range(len(answer)):
        for j in range(len(answer[i])):
            
            # 아래에 for문이 가장 핵심이다. 
            for k in range(len(arr1[i])):
                if arr1[i][k] + arr2[k][j] >=1: #조건식 입력
                    answer[i][j] += 1

    return answer


### 두 문서 사이의 합집합 계산

In [275]:
df_union=pd.DataFrame(np.array(solution(a, b)),index=doc,columns=doc)
df_union


,A,B,C,D
A,5,7,6,8
B,7,4,6,6
C,6,6,3,6
D,8,6,6,3


## jaccard 유사도 계산하기

In [281]:
jac_result=a.dot(b)/np.array(solution(a, b))
df_result=pd.DataFrame(jac_result,index=doc,columns=doc)
df_result

,A,B,C,D
A,1.000000,0.285714,0.333333,0.000000
B,0.285714,1.000000,0.166667,0.166667
C,0.333333,0.166667,1.000000,0.000000
D,0.000000,0.166667,0.000000,1.000000


## 결과 저장하기

In [282]:
writer=pd.ExcelWriter("./jacc_result_custom.xlsx")

In [283]:
df_intersection.to_excel(writer,sheet_name="sheet1")
df_union.to_excel(writer,sheet_name="sheet2")
df_result.to_excel(writer,sheet_name="sheet3")
writer.save()